## Set-Up Phase

Many thanks to the authors of this tutorial, who taught me how to create a deep learning model: https://medium.com/@sssspppp/image-classification-using-cnn-0fad8367acfd

In [ ]:
# Install all relevant tools
!pip install tensorflow tensorflow-gpu opencv-python matplotlib
!pip list
!pip install ucimlrepo
!pip install keras

  Using cached tensorflow-gpu-2.12.0.tar.gz (2.6 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
Package                            Version
---------------------------------- -------------------
absl-py                            1.4.0
accelerate                         0.34.2
aiohappyeyeballs                   2.4.3
aiohttp                            3.10.10
aiosignal                          1.3.1
alabaster                          0.7.16
albucore                           0.0.19
albumentations                     1.4.20
altair                        

In [ ]:
# Import Libraries
import cv2
import tensorflow as tf
import os
import imghdr
import numpy as np
import kagglehub
from matplotlib import pyplot as plt
# Used to do cv2.imshow in Google Collab
from google.colab.patches import cv2_imshow
import json
import shutil

## Loading Data

In [ ]:
# Used command from kaggle instructions on how to download data -https://www.kaggle.com/datasets/nuttidalapthanachai/cloud-image-dataset

path = kagglehub.dataset_download("nuttidalapthanachai/cloud-image-dataset")

In [ ]:
# Check paths and contents of files
print(os.listdir(path + "/train/Cloud"))

['cloud548.jpg', 'cloud429.jpg', 'cloud482.jpg', 'cloud506.jpg', 'cloud165.jpg', 'cloud274.jpg', 'cloud233.jpg', 'cloud296.jpg', 'cloud497.jpg', 'cloud206.jpg', 'cloud142.jpg', 'cloud225.jpg', 'cloud208.jpg', 'cloud328.jpg', 'cloud343.jpg', 'cloud254.jpg', 'cloud494.jpg', 'cloud554.jpg', 'cloud114.jpg', 'cloud592.jpg', 'cloud458.jpg', 'cloud177.jpg', 'cloud462.jpg', 'cloud201.jpg', 'cloud171.jpg', 'cloud595.jpg', 'cloud323.jpg', 'cloud247.jpg', 'cloud565.jpg', 'cloud351.jpg', 'cloud285.jpg', 'cloud517.jpg', 'cloud287.jpg', 'cloud465.jpg', 'cloud463.jpg', 'cloud556.jpg', 'cloud252.jpg', 'cloud152.jpg', 'cloud404.jpg', 'cloud248.jpg', 'cloud393.jpg', 'cloud130.jpg', 'cloud466.jpg', 'cloud245.jpg', 'cloud479.jpg', 'cloud200.jpg', 'cloud191.jpg', 'cloud315.jpg', 'cloud564.jpg', 'cloud299.jpg', 'cloud504.jpg', 'cloud534.jpg', 'cloud424.jpg', 'cloud481.jpg', 'cloud155.jpg', 'cloud217.jpg', 'cloud410.jpg', 'cloud519.jpg', 'cloud263.jpg', 'cloud258.jpg', 'cloud432.jpg', 'cloud359.jpg', 'cloud2

In [ ]:
# Load images for training
training_data = tf.keras.utils.image_dataset_from_directory(path + '/train/')
data_iterator_train = training_data.as_numpy_iterator()

# Testing the image has loaded correctly
# batch = data_iterator.next()
# fig, ax = plt.subplots(ncols=4, figsize=(20, 20))
# for idx, img in enumerate(batch[0][:4]):
#   ax[idx].imshow(img.astype(int))
#   ax[idx].title.set_text(batch[1][idx])

# Separating data into training and validating data (testing data is separate)
train_size = int(len(training_data) * .95)
val_size = int(len(training_data) * .05)
train = data.take(train_size)
val = data.skip(train_size).take(val_size)

Found 1000 files belonging to 2 classes.


In [ ]:
# Load image test data
testing_data = tf.keras.utils.image_dataset_from_directory(path + '/test/')
data_iterator_test = testing_data.as_numpy_iterator()
test = testing_data.take(len(testing_data))

# Testing images have loaded correctly
# batch = data_iterator.next()
# fig, ax = plt.subplots(ncols=4, figsize=(20, 20))
# for idx, img in enumerate(batch[0][:4]):
#   ax[idx].imshow(img.astype(int))
#   ax[idx].title.set_text(batch[1][idx])

Found 200 files belonging to 2 classes.


## Building and Training Deep Learning Model

In [ ]:
# Building a deep learning model

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout

model = Sequential()

# Experimented with number of kernels and layers (initially had 4 layers,
# but moved to 2 after overfitting occurred)

# Layer 1
model.add(Conv2D(12, (3,3), 1, activation='relu', input_shape=(256, 256, 3)))
# Layer 2 - condenses image
model.add(MaxPooling2D())

# Flatten data
model.add(Flatten())

# Dense layers
model.add(Dense(256, activation='relu')) # as stated in tutorial, this output is 256 values

# Sigmoid layer - 0 is cloud, 1 is rain cloud
model.add(Dense(1, activation='sigmoid')) # produces values between 0 and 1

# Use ADAM optimiser function and check accuracy
model.compile('adam', loss=tf.losses.BinaryCrossentropy(), metrics=['accuracy'])

# Look at statistics
model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_12 (Conv2D)                   │ (None, 254, 254, 12)        │             336 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_12 (MaxPooling2D)      │ (None, 127, 127, 12)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_5 (Flatten)                  │ (None, 193548)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ (None, 256)                 │      49,548,544 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ (None, 1)                   │             257 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 49,549,137 (189.01 MB)

 Trainable params: 49,549,137 (189.01 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Training model on data
# Note to self - the more epochs, not necessarily better

hist = model.fit(train, epochs=2, validation_data=val)
hist.history

Epoch 1/2
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.5909 - loss: 4477.8887

30/30 ━━━━━━━━━━━━━━━━━━━━ 64s 2s/step - accuracy: 0.5942 - loss: 4391.9243 - val_accuracy: 0.9062 - val_loss: 45.3983
Epoch 2/2
30/30 ━━━━━━━━━━━━━━━━━━━━ 64s 2s/step - accuracy: 0.8886 - loss: 149.3446 - val_accuracy: 0.8438 - val_loss: 236.5469


{'accuracy': [0.6927083134651184, 0.878125011920929],
 'loss': [1812.995361328125, 149.85704040527344],
 'val_accuracy': [0.90625, 0.84375],
 'val_loss': [45.398277282714844, 236.54693603515625]}

In [ ]:
# Saving model to be used - this is important
# IF YOU SIMPLY WANT TO CREATE THE TRAINED MODEL, THIS IS THE LAST INSTRUCTION YOU NEED TO RUN
model.save('/content/trained_model.keras')

## Testing Stage

In [ ]:
# Loading model from saved file
import keras
model_loaded = keras.models.load_model('/content/trained_model.keras')

In [ ]:
# Testing
image_names = os.listdir(path + "/test/Cloud/")
print(image_names)

cloud_accuracy = 0
for x in image_names:
  img = cv2.imread(path + "/test/Cloud/" + str(x))
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  resize = tf.image.resize(img, (256, 256))
  np.expand_dims(resize, 0)
  test_prediction = model_loaded.predict(np.expand_dims(resize/255, 0))
  if test_prediction > 0.5:
    print("It is a rain cloud.")
  else:
    print("It is a cloud")
    cloud_accuracy += 1

print("CLOUD ACCURACY:" + str(cloud_accuracy/len(image_names)))

image_names = os.listdir(path + "/test/RainCloud/")
print(image_names)

rain_cloud_accuracy = 0

for x in image_names:
  img = cv2.imread(path + "/test/RainCloud/" + str(x))
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  resize = tf.image.resize(img, (256, 256))
  np.expand_dims(resize, 0)
  test_prediction = model_loaded.predict(np.expand_dims(resize/255, 0))
  if test_prediction > 0.5:
    print("It is a rain cloud.")
    rain_cloud_accuracy += 1
  else:
    print("It is a cloud")

print("RAIN CLOUD ACCURACY:" + str(rain_cloud_accuracy/len(image_names)))



['cloud76.jpg', 'cloud60.jpg', 'cloud18.jpg', 'cloud35.jpg', 'cloud57.jpg', 'cloud94.jpg', 'cloud45.jpg', 'cloud71.jpg', 'cloud55.jpg', 'cloud67.jpg', 'cloud96.jpg', 'cloud20.jpg', 'cloud72.jpg', 'cloud19.jpg', 'cloud12.jpg', 'cloud88.jpg', 'cloud74.jpg', 'cloud17.jpg', 'cloud70.jpg', 'cloud93.jpg', 'cloud13.jpg', 'cloud10.jpg', 'cloud14.jpg', 'cloud37.jpg', 'cloud58.jpg', 'cloud28.jpg', 'cloud42.jpg', 'cloud92.jpg', 'cloud56.jpg', 'cloud36.jpg', 'cloud89.jpg', 'cloud51.jpg', 'cloud68.jpg', 'cloud5.jpg', 'cloud99.jpg', 'cloud8.jpg', 'cloud90.jpg', 'cloud29.jpg', 'cloud34.jpg', 'cloud98.jpg', 'cloud7.jpg', 'cloud4.jpg', 'cloud40.jpg', 'cloud9.jpg', 'cloud65.jpg', 'cloud79.jpg', 'cloud87.jpg', 'cloud30.jpg', 'cloud62.jpg', 'cloud52.jpg', 'cloud49.jpg', 'cloud15.jpg', 'cloud81.jpg', 'cloud59.jpg', 'cloud53.jpg', 'cloud26.jpg', 'cloud22.jpg', 'cloud11.jpg', 'cloud54.jpg', 'cloud25.jpg', 'cloud2.jpg', 'cloud66.jpg', 'cloud47.jpg', 'cloud91.jpg', 'cloud1.jpg', 'cloud100.jpg', 'cloud21.jpg', 